In [1]:
import os
# 获取当前文件所在目录
current_dir = os.path.dirname(os.path.abspath(''))
print("当前工作目录:", current_dir)
# 设置工作目录
os.chdir('/data/zlhu/NeuroAI/Eye-of-Brain/Brain-Diffusion/alignment')
# 验证工作目录
print("当前工作目录:", os.getcwd())

当前工作目录: /data
当前工作目录: /data/zlhu/NeuroAI/Eye-of-Brain/Brain-Diffusion/alignment


In [2]:
import argparse, os
import numpy as np
from himalaya.backend import set_backend
from himalaya.ridge import RidgeCV
from himalaya.scoring import correlation_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [3]:
# 参数设置
source='subj07'
target = 'subj05'
roi = ['early']   # 需要对齐的ROI
backend = set_backend("numpy", on_error="warn")

In [4]:
# 路径设置
sourdir = f'../data/fmri/{source}'
tardir = f'../data/fmri/{target}'
savedir = f'../data/fmri_align/{target}'
# os.makedirs(savedir, exist_ok=True)

In [5]:
# 超参数选择，正则化系数
# alpha = [1e3, 3e3, 6e3, 9e3, 1e4, 3e4, 6e4, 9e4, 1e5]
alpha = [0.000001,0.00001,0.0001,0.001,0.01, 0.1, 1]

In [6]:
'''算法流程'''
# 创建岭回归
ridge = RidgeCV(alphas=alpha)

# 创建预处理流程
preprocess_pipeline = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
)

# 预处理+岭回归 流程pipeline
pipeline = make_pipeline(
    preprocess_pipeline,
    ridge,
) 

In [7]:
'''加载训练和测试数据'''
X = []  # 所有对齐样本 1000（800：200）

for croi in roi:
    cX = np.load(f'{tardir}/{target}_{croi}_betas_ave_te.npy').astype("float32")
    X.append(cX)

# 堆叠所有roi并转为np
X = np.hstack(X)

X_tr = X[:800]
X_te = X[800:]


Y = []   
for croi in roi:
    cY = np.load(f'{sourdir}/{source}_{croi}_betas_ave_te.npy').astype("float32")
    Y.append(cY)

# 目标数据（图像特征）
Y = np.hstack(Y)

Y_tr = Y[:800]
Y_te = Y[800:]

In [8]:
print(f'X_tr {X_tr.shape}, X_te {X_te.shape}, Y_tr {Y_tr.shape}, Y_te {Y_te.shape}')

X_tr (800, 4494), X_te (200, 4494), Y_tr (800, 4559), Y_te (200, 4559)


In [9]:
# 训练对齐模型
print(f'Now making alignment model for... {target} -align-to- {source} in {roi}')
pipeline.fit(X_tr, Y_tr)

Now making alignment model for... subj05 -align-to- subj07 in ['early']


/data/zlhu/miniconda3/envs/brain/lib/python3.8/site-packages/himalaya/ridge/_random_search.py:491: UserWarning: Solving ridge is slower than solving kernel ridge when n_samples < n_features (here 800 < 4494). Using a linear kernel in himalaya.kernel_ridge.KernelRidgeCV or himalaya.kernel_ridge.solve_kernel_ridge_cv_eigenvalues would be faster. Use warn=False to silence this warning.
  warnings.warn(


Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('standardscaler', StandardScaler())])),
                ('ridgecv',
                 RidgeCV(alphas=[1e-06, 1e-05, 0.0001, 0.001, 0.01, 0.1, 1]))])

In [10]:
# 模型预测和评估
brain_align = pipeline.predict(X_te)
rs = correlation_score(Y_te.T,brain_align.T)
print(f'Prediction accuracy is: {np.mean(rs):3.3}')

Prediction accuracy is: 0.0465


In [11]:
print(X_te[0])
print(brain_align[0])
print(Y_te[0])
print(brain_align.shape)

[299.33334   58.333332 797.3333   ...  29.333334 -60.       -81.      ]
[ 343.23886   858.7257    457.3103   ...  -13.167908 -110.56087
 -202.75308 ]
[ -425.33334 -1332.6666   -223.      ...  -684.3333  -1225.6666
  -429.33334]
(200, 4559)


In [12]:
best_alpha = pipeline.named_steps['ridgecv'].best_alphas_
print("Best alpha parameter:", best_alpha.mean())

Best alpha parameter: 1.0


In [13]:
# 保存预测结果
np.save(f'{savedir}/{target}_{"_".join(roi)}_brain_align2_{source}.npy',brain_align)